# M2.1 Tensorboard on Goolge Colab



In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-10-30 12:12:58--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.237.103, 52.1.65.199, 34.199.255.1, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.237.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  7.70MB/s    in 0.7s    

2018-10-30 12:13:00 (7.70 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## CNN Prediction
- Read Training FIle
- Build Model
- Train Model
- Predict Model

In [0]:
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DATA_PATH = "/content/gdrive/My Drive/Lecture/StudyPie/Data/"

In [0]:
!ls "/content/gdrive/My Drive/Lecture/StudyPie/Data/"

256x2-CNN.model  crypto_data	  M2_1_X.pickle  __MACOSX   PetImages.zip
CoinOne		 crypto_data.zip  M2_1_y.pickle  PetImages


In [0]:
# Read training file
import pickle

pickle_in = open(DATA_PATH+"M2_1_X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open(DATA_PATH+"M2_1_y.pickle","rb")
y = pickle.load(pickle_in)

In [0]:
X = X/255.0

TRAIN_RATIO = 0.8

X_train = X[:int(len(X)*TRAIN_RATIO)]
y_train = y[:int(len(y)*TRAIN_RATIO)]

X_test = X[int(len(X)*TRAIN_RATIO):]
y_test = y[int(len(y)*TRAIN_RATIO):]

## Run TensorBoard

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

## Run ngrok

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL

In [0]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://9002ba4c.ngrok.io


## Run a Keras model with TenorBoard

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         write_images=True)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=5, validation_split=0.3,
          callbacks=[tbCallBack])

Train on 3080 samples, validate on 1321 samples
Epoch 1/5
3080/3080 [==============================] - 325s 106ms/step - loss: 0.7826 - acc: 0.5237 - val_loss: 0.6872 - val_acc: 0.5602


## Optionally check tensorboard and ngrok PIDs and stop them

In [0]:
#!ps aux | grep tensorboard

root         986  0.9  3.3 1729856 445480 ?      Sl   07:07   0:04 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./log --host 0.0.0.0 --port 6006
root        1060  0.0  0.0  39196  6528 ?        S    07:15   0:00 /bin/bash -c ps aux | grep tensorboard
root        1062  0.0  0.0  39196  2696 ?        D    07:15   0:00 /bin/bash -c ps aux | grep tensorboard


In [0]:
#!kill 986

## Optionally clear the tensorboard records directory

In [0]:
#!rm -rf log